baizer::pkglib(baizer, tidyverse, jsonlite, ComplexHeatmap)

sample <- snakemake@wildcards[['sample']]

# replace default configs with sample configs
config <- replace_item(snakemake@config, snakemake@config[[sample]])

# mRNA filter
mRNA_gene_flt <- config[['mRNA_gene_flt']]
mRNA_umi_flt <- config[['mRNA_umi_flt']]
mRNA_mt_percent_flt <- config[['mRNA_mt_percent_flt']]
VDJB_umi_flt <- config[['VDJB_umi_flt']]
VDJT_umi_flt <- config[['VDJT_umi_flt']]

Pstat_dir <- snakemake@input[['stat_dir']]
Pfilter_dir <- snakemake@output[['filter_dir']]
dir.create(Pfilter_dir, recursive = TRUE)

subdirs <- dir(Pstat_dir)

# mRNA
if ('mRNA' %in% subdirs) {
    mRNA <- read_csv(str_glue('{Pstat_dir}/mRNA/mRNA.csv'))

    mRNA <- mRNA %>% 
        mutate(across(c(nFeature_RNA, nCount_RNA, mt_percent), ~ifelse(is.na(.x), 0, .x))) %>%
        mutate(
            mRNA_gene_keep = nFeature_RNA > mRNA_gene_flt,
            mRNA_umi_keep = nCount_RNA > mRNA_umi_flt,
            mRNA_mt_percent_keep = mt_percent < mRNA_mt_percent_flt,
            mRNA_keep = all(mRNA_gene_keep, mRNA_umi_keep, mRNA_mt_percent_keep)
        )
    write_excel_csv(mRNA %>% filter(mRNA_keep==TRUE), str_glue('{Pfilter_dir}/mRNA_flt.csv'))
}

# VDJB
if ('VDJB' %in% subdirs) {
    VDJB <- read_csv(str_glue('{Pstat_dir}/VDJB/VDJB.csv'))
    
    VDJB <- VDJB %>% 
        mutate(across(c(umis_H, umis_L), ~ifelse(is.na(.x), 0, .x))) %>%
        mutate(
            VDJB_productive_igblast_keep = productive_igblast,
            VDJB_productive_cellranger_keep = productive_cellranger,
            VDJB_unique_keep = unique, 
            VDJB_umi_H_keep = umis_H > VDJB_umi_flt,
            VDJB_umi_L_keep = umis_L > VDJB_umi_flt,
            VDJB_keep = all(VDJB_productive_igblast_keep, VDJB_productive_cellranger_keep, VDJB_unique_keep,
                           VDJB_umi_H_keep, VDJB_umi_L_keep)
        )
    write_excel_csv(VDJB %>% filter(VDJB_keep==TRUE), str_glue('{Pfilter_dir}/VDJB_flt.csv'))
}

# VDJT
if ('VDJT' %in% subdirs) {
    VDJT <- read_csv(str_glue('{Pstat_dir}/VDJT/VDJT.csv'))
    
    VDJT <- VDJT %>% 
        mutate(across(c(umis_H, umis_L), ~ifelse(is.na(.x), 0, .x))) %>%
        mutate(
            VDJT_productive_cellranger_keep = productive_cellranger,
            VDJT_unique_keep = unique, 
            VDJT_umi_H_keep = umis_H > VDJT_umi_flt,
            VDJT_umi_L_keep = umis_L > VDJT_umi_flt,
            VDJT_keep = all(VDJT_productive_cellranger_keep, VDJT_unique_keep,
               VDJT_umi_H_keep, VDJT_umi_L_keep)
        )
    write_excel_csv(VDJT %>% filter(VDJT_keep==TRUE), str_glue('{Pfilter_dir}/VDJT_flt.csv'))
}